## **Set up working directory and install dependencies**

In [1]:
# Set up working directory
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
# Install modrdred pacakge
!pip install mordred ##version 1.2.0
!pip install rdkit  ##version 2024.9.6

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.4/36.4 MB 28.6 MB/s eta 0:00:00


In [9]:
# Libraries to help with reading and manipulating data
import numpy as np
import pandas as pd
# import Chemoinformatic libraries
from rdkit import Chem
from rdkit.Chem import AllChem, Descriptors, Draw, PandasTools

To predict AhR activity for phenanthrene, 7H-Benzo[c]carbazole, and Benzo[k]fluoranthene as an example

*   Retrieve its 3D SDF from the PubChem database using the CID number. A CSV file can be uploaded for batch retrieval.
*   Calculate the moelcular descriptor
*   Predict their AhR activity (1: active; 0: inactive)
*   Predict thier log (EC10 values (μM)) values





## **Molecular descriptor calculation**

In [22]:
# read 3D SDF files for chemicals
df = PandasTools.LoadSDF('/content/3D_sdf_example.sdf')
suppl = Chem.SDMolSupplier('/content/3D_sdf_example.sdf')

In [23]:
#Related files to annotate molecular descriptor name and filter molecular descriptors
Data_mordred_name = pd.read_csv('/content/mordred descriptors.csv', names = ['descriptors'])
descriptor_name = Data_mordred_name['descriptors'].to_list()
af = pd.read_csv('/content/Training_data_classification_selected_descriptors.csv', index_col = 0)
af_without_labels = af.drop(columns = ['Label'])

In [24]:
CID = df['PUBCHEM_COMPOUND_CID'].to_list()

from mordred import Calculator, descriptors
mordred_calc = Calculator(descriptors, ignore_3D=False)

# The deprecation for the aliases np.object, np.bool, np.float, np.complex, np.str, and np.int is expired (introduces NumPy 1.20)
np.float = float
np.int = int          #module 'numpy' has no attribute 'int'
np.object = object    #module 'numpy' has no attribute 'object'
np.bool = bool        #module 'numpy' has no attribute 'bool'


result = []
for mol in suppl:
  des = mordred_calc(mol)
  result.append(des)
df_mordred = pd.DataFrame(result, index = CID)

In [25]:
## rename columns using the list
df_mordred.columns = descriptor_name

## fitler columns to use 529 molecular descriptors
numeric_columns = af_without_labels.columns
df_mordred_filter = df_mordred[numeric_columns]

In [26]:
## Using MinMaxScaler with values below the minimum from the fitting dataset will scale them accordingly, producing values less than 0.
## Handle these out-of-bound values by clipping

# To scale the data from 0-1
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

##-----------------------------------------------
## The fit() method is called on training dataset, which computes and stores the minimum and maximum values of each feature in each column
scaler = scaler.fit(af_without_labels)

df_scaled = scaler.transform(df_mordred_filter)
df_scaled = pd.DataFrame(df_scaled, columns = df_mordred_filter.columns, index = df_mordred_filter.index)

In [27]:
df_scaled

,ABC,nAcid,nBase,SpMax_A,SpMAD_A,VE1_A,VR1_A,nAromAtom,nAtom,nSpiro,...,JGI6,JGI7,JGI8,JGI9,JGI10,JGT10,TopoShapeIndex,SRW05,TSRW10,mZagreb1
995,0.244247,0.0,0.0,0.570486,0.955185,0.421066,0.001358,0.466667,0.198113,0.0,...,0.320000,0.000000,0.000000,0.0,0.0,0.272484,0.75,0.000000,0.304114,0.090961
67459,0.312877,0.0,0.0,0.625842,0.959793,0.482226,0.002063,0.566667,0.235849,0.0,...,0.118788,0.274510,0.000000,0.0,0.0,0.283598,1.00,0.452151,0.500064,0.110535
9158,0.381507,0.0,0.0,0.658192,0.987860,0.538039,0.002979,0.666667,0.273585,0.0,...,0.200041,0.168334,0.086097,0.0,0.0,0.298890,0.60,0.452151,0.546932,0.130109


## **Binary classification model to Predict AhR activity**

In [28]:
from google.colab import files
import joblib

In [29]:
# Download the trained model
xgb_classify = joblib.load('Classification_model_20250918.pkl')

In [30]:
# First calculate the probability of each molecular structure and use threshold of 0.446 to classifiy the activity. 1: above 0.446 and 0: below 0.446
predicted_proba = xgb_classify.predict_proba(df_scaled)
Activity_prediction = (predicted_proba[:,1] >= 0.446).astype('int')

In [31]:
Activity_prediction

array([0, 1, 1])

## **Regression model to Predict AhR activity**

In [32]:
# Download the trained model
xgb_regressor = joblib.load('/content/Regression_model_20251104.pkl')

In [33]:
# Predict log (EC10 values (μM)) for each molecular structure
potency_prediction = xgb_regressor.predict(df_scaled)

In [34]:
potency_prediction

array([ 0.92046744, -0.54340875, -2.3582406 ], dtype=float32)